In [1]:
# Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

2023-06-28 09:30:01.086060: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 09:30:01.159289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 09:30:01.160522: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 09:30:02.171877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df=pd.read_csv(r"https://raw.githubusercontent.com/krishnaik06/Keras-Tuner/main/Real_Combine.csv")

In [3]:
df

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [4]:
df.isnull().sum()
df.dropna(inplace=True,axis=0)
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [5]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

# Hyperparameters
- How many number of hidden layers we should have?
- How many number of neurons we should have in hidden layers?
- Learning Rate

In [6]:

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [7]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Tuner from project/Air Quality Index/tuner0.json


In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [10]:
tuner.search(X_train, y_train,
             epochs=50,
             validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 4 summary
Hyperparameters:
num_layers: 11
units_0: 448
units_1: 320
learning_rate: 0.01
units_2: 384
units_3: 32
units_4: 384
units_5: 160
units_6: 512
units_7: 416
units_8: 160
units_9: 320
units_10: 384
units_11: 128
units_12: 64
units_13: 384
units_14: 320
units_15: 224
units_16: 160
units_17: 288
Score: 59.11651484171549

Trial 1 summary
Hyperparameters:
num_layers: 3
units_0: 416
units_1: 64
learning_rate: 0.001
units_2: 32
Score: nan

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 416
units_1: 64
learning_rate: 0.0001
Score: nan

Trial 2 summary
Hyperparameters:
num_layers: 6
units_0: 256
units_1: 480
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32

Trial 3 summary
Hyperparameters:
num_layers: 18
units_0: 256
units_1: 416
learning_rate: 0.0001
units_2: 224
units_3: 224
units_4: 64
units_5: 32
units_6: 32
units_7

In [12]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

In [14]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 448, kernel_initializer = 'he_uniform',activation='relu',input_dim = 8))
# classifier.add(Dropout(0.3))
# Adding the second hidden layer
classifier.add(Dense(units = 320, kernel_initializer = 'he_uniform',activation='relu'))
classifier.add(Dense(units = 384, kernel_initializer = 'he_uniform',activation='relu'))

# classifier.add(Dropout(0.2))

classifier.add(Dense(units = 32, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.1))

classifier.add(Dense(units = 384, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.4))

classifier.add(Dense(units =160, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.3))

classifier.add(Dense(units = 512, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.4))

classifier.add(Dense(units = 416, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.3))
# Adding the second hidden layer
classifier.add(Dense(units = 160, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.4))
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 320, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.3))
# Adding the second hidden layer
classifier.add(Dense(units = 384, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.4))

classifier.add(Dense(units = 128, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.3))

classifier.add(Dense(units = 162, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.4))

classifier.add(Dense(units = 64, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.3))

classifier.add(Dense(units = 384, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.4))

classifier.add(Dense(units = 320, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.3))

classifier.add(Dense(units = 224, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.4))

classifier.add(Dense(units = 160, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.3))

classifier.add(Dense(units = 288, kernel_initializer = 'he_uniform',activation='relu'))
# classifier.add(Dropout(0.4))

# Adding the output layer
classifier.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='linear'))

In [15]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 448)               4032      
                                                                 
 dense_1 (Dense)             (None, 320)               143680    
                                                                 
 dense_2 (Dense)             (None, 384)               123264    
                                                                 
 dense_3 (Dense)             (None, 32)                12320     
                                                                 
 dense_4 (Dense)             (None, 384)               12672     
                                                                 
 dense_5 (Dense)             (None, 160)               61600     
                                                                 
 dense_6 (Dense)             (None, 512)               8

In [16]:
classifier.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [17]:
# Fitting the ANN to the Training set
model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 500, epochs = 300)

Epoch 1/300


/opt/conda/lib/python3.10/site-packages/keras/engine/data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


2/2 [==============================] - 4s 344ms/step - loss: 20268.4355 - mean_absolute_error: 113.8237 - val_loss: 10223.1309 - val_mean_absolute_error: 74.1958
Epoch 2/300
2/2 [==============================] - 0s 57ms/step - loss: 11603.1582 - mean_absolute_error: 77.7214 - val_loss: 5838.7114 - val_mean_absolute_error: 58.9730
Epoch 3/300
2/2 [==============================] - 0s 58ms/step - loss: 6521.9097 - mean_absolute_error: 59.2592 - val_loss: 10323.0225 - val_mean_absolute_error: 76.2694
Epoch 4/300
2/2 [==============================] - 0s 59ms/step - loss: 11515.3564 - mean_absolute_error: 78.8515 - val_loss: 7013.6782 - val_mean_absolute_error: 61.8300
Epoch 5/300
2/2 [==============================] - 0s 78ms/step - loss: 7588.5679 - mean_absolute_error: 61.4648 - val_loss: 12217.6621 - val_mean_absolute_error: 85.3171
Epoch 6/300
2/2 [==============================] - 0s 65ms/step - loss: 10359.1777 - mean_absolute_error: 80.8043 - val_loss: 4808.7861 - val_mean_absolut

In [18]:
y_pred = classifier.predict(X_test)

11/11 [==============================] - 0s 4ms/step


In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
# Assuming y_true contains the true target values and y_pred contains the predicted values
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [20]:
r2 

0.6101160862347526

In [21]:
rmse

52.84932282590104